### Reinitialize the file

In [1]:
# warning !! remove all except this script
## remove -i argument to run
!rm `(ls | grep -v '.ipynb')`

rm: cannot remove 'demo_transform_structure': Is a directory


In [1]:
pdbid='2jpz'
drug='EWV'
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid

In [3]:
import os
ff_folder  = os.path.realpath("../../../smsl/opt/force_field")
mdp_folder = os.path.realpath("../../../smsl/opt/mdp")

### G-quadruplex Prepare

In [4]:
!cp ../../c.gromacs.run_md/1.create_system/*{drug}* ./
!cp ../../c.gromacs.run_md/1.create_system/*.itp ./
!cp ../../c.gromacs.run_md/1.create_system/*.top ./
!cp ../../c.gromacs.run_md/1.create_system/{pdbid}.gro.pdb ./
!cp ../5.Output/{jobname}.gro.pdb ./

In [5]:
# 畫完box後座標會平移
# 因此最後要將座標再align回原本座標
# 因為不建立水箱 因此 -d 數值隨意(但不要太小以防止因為pbc而導致結構被切一半) 
!gmx editconf -f {jobname}.gro.pdb -o {jobname}_dode.pdb -c yes -d 1.5 -bt dodecahedron

                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_ewv/b.amber.drug_ff/6.Em_min
Command line:
  gmx editconf -f 2jpz_EWV.gro.pdb -o 2jpz_EWV_dode.pdb -c yes -d 1.5 -bt dodecahedron

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 760 atoms
No velocities found
    system size :  2.520  2.938  2.445 (nm)
    diameter    :  3.084               (nm)
    center      :  0.012 -0.060 -0.069 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  4.551  4.623  2.220 (nm)
new center      :  4.563  4.563  2.151 (nm)
new box vectors :  6.084  6.084  6.084 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 159.25               (nm^3)

GROMACS reminds you: "So ca

In [6]:
cmd = f'''\
r EWV | r DG
name 5 {drug}_DG
q
'''
!echo -e "{cmd}" | gmx make_ndx -f {jobname}.gro.pdb -o {jobname}.ndx

                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_ewv/b.amber.drug_ff/6.Em_min
Command line:
  gmx make_ndx -f 2jpz_EWV.gro.pdb -o 2jpz_EWV.ndx


Reading structure file
Going to read 0 old index file(s)
Analysing residue names:
There are:    22        DNA residues
There are:     2        Ion residues
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              :   760 atoms
  1 DNA                 :   715 atoms
  2 K                   :     2 atoms
  3 EWV                 :    43 atoms
  4 Ion                 :     2 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr   

### 因為藥物跟loop空間上會重疊，因此將藥物往下位移一點

In [2]:
cmd=f'''\
pymol >>
load {jobname}_dode.pdb
extract {drug}, resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele {drug}
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save {jobname}_dode.1.pdb, enable
>> EOF
'''
print(cmd)

pymol >>
load 2jpz_EWV_dode.pdb
extract EWV, resn EWV
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele EWV
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save 2jpz_EWV_dode.1.pdb, enable
>> EOF



## First Cycle of EM1

In [8]:
## 固定藥物和核心區域(DG)
freeze_cmd = f'''\
freezegrps      = {drug}_DG
freezedim       = Y Y Y
'''

In [9]:
!echo "{freeze_cmd}"        > min1.mdp
!cat {mdp_folder}/min1.mdp >> min1.mdp

In [10]:
!gmx grompp -f min1.mdp -c {jobname}_dode.1.pdb -r {jobname}_dode.1.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_ewv/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min1.mdp -c 2jpz_EWV_dode.1.pdb -r 2jpz_EWV_dode.1.pdb -p topol.top -o min1.tpr -n 2jpz_EWV.ndx -maxwarn 1

Setting the LD random seed to -579903499

Generated 4851 of the 4851 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4851 of the 4851 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'EWV'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -18.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 

### First cycle of EM2

In [11]:
!echo "{freeze_cmd}"        > min2.mdp
!cat {mdp_folder}/min1.mdp >> min2.mdp

In [12]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.1.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_ewv/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min2.mdp -c min1.pdb -r 2jpz_EWV_dode.1.pdb -p topol.top -o min2.tpr -n 2jpz_EWV.ndx -maxwarn 1

Setting the LD random seed to 2092740450

Generated 4851 of the 4851 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4851 of the 4851 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'EWV'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -18.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1 [file top

### Second cycle
### 現在上層loop往上移一點了，因此我們可以再將藥物往上移一點

In [3]:
cmd=f'''\
pymol >>
load min2.pdb
extract {drug}, resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele {drug}
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save {jobname}_dode.2.pdb, enable
>> EOF
'''
print(cmd)

pymol >>
load min2.pdb
extract EWV, resn EWV
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele EWV
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save 2jpz_EWV_dode.2.pdb, enable
>> EOF



### Second cycle of EM1

In [14]:
!gmx grompp -f min1.mdp -c {jobname}_dode.2.pdb -r {jobname}_dode.2.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_ewv/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min1.mdp -c 2jpz_EWV_dode.2.pdb -r 2jpz_EWV_dode.2.pdb -p topol.top -o min1.tpr -n 2jpz_EWV.ndx -maxwarn 1

Setting the LD random seed to 2029877239

Generated 4851 of the 4851 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4851 of the 4851 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'EWV'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -18.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 

### Second cycle of EM2

In [15]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.2.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_ewv/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min2.mdp -c min1.pdb -r 2jpz_EWV_dode.2.pdb -p topol.top -o min2.tpr -n 2jpz_EWV.ndx -maxwarn 1

Setting the LD random seed to -275802113

Generated 4851 of the 4851 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4851 of the 4851 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'EWV'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -18.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1 [file top

### Third cycle
### 現在將藥物換成原本的pose再做最後一次能量最小化

In [16]:
cmd=f'''\
pymol >>
load min2.pdb
extract {drug}_min2, min2 and resn {drug}
load {jobname}_dode.pdb
extract {drug}, {jobname} and resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
set retain_order, 1
save {jobname}_dode.3.pdb, min2 or {drug}
>> EOF
'''
print(cmd)

pymol >>
load min2.pdb
extract EWV_min2, min2 and resn EWV
load 2jpz_EWV_dode.pdb
extract EWV, 2jpz_EWV and resn EWV
set retain_order, 1
save 2jpz_EWV_dode.3.pdb, min2 or EWV
>> EOF



### Third cycle of EM1

In [17]:
!gmx grompp -f min1.mdp -c {jobname}_dode.3.pdb -r {jobname}_dode.3.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_ewv/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min1.mdp -c 2jpz_EWV_dode.3.pdb -r 2jpz_EWV_dode.3.pdb -p topol.top -o min1.tpr -n 2jpz_EWV.ndx -maxwarn 1

Setting the LD random seed to 670562111

Generated 4851 of the 4851 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4851 of the 4851 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'EWV'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -18.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1

### Second cycle of EM2

In [18]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.2.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_ewv/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min2.mdp -c min1.pdb -r 2jpz_EWV_dode.2.pdb -p topol.top -o min2.tpr -n 2jpz_EWV.ndx -maxwarn 1

Setting the LD random seed to 1828698107

Generated 4851 of the 4851 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4851 of the 4851 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'EWV'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -18.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1 [file top

In [19]:
cmd=f'''\
pymol >>
load min2.pdb
load {jobname}.gro.pdb
align min2, {jobname}.gro
set retain_order, 1
save {jobname}.final.gro.pdb, min2
>> EOF
'''
print(cmd)

pymol >>
load min2.pdb
load 2jpz_EWV.gro.pdb
align min2, 2jpz_EWV.gro
set retain_order, 1
save 2jpz_EWV.final.gro.pdb, min2
>> EOF



最終 {jobname}.final.gro.pdb 就是分子模擬的初始結構